In [1]:
import rdflib

# Querying the Knowledge Graph

Testing the KG and ontology with SPARQL built from the requirements and competency questions.


|Identifier|Requirement or Competency Question                                                                                           |
|----------|-----------------------------------------------------------------------------------------------------------------------------|
|R1        |Articles have title, year, journal, issue, abstract, volume and author                                                       |
|R2        |Each article may refer to one or more experiments                                                                            |
|R3        |Each experiment must have a catalyst                                                                                         |
|R4        |Each experiment may have additional inputs apart from the catalyst                                                           |
|R5        |Each experiment must specify surface area and band gap                                                                       |
|R6        |An experiment must indicate operation conditions, including temperature, pressure, reaction time, medium and operation mode  |
|R7        |Photocatalysis experiments may also indicate type of reactor                                                                 |
|R8        |Electrocatalysis and Photoelectrocatalysis experiments may also indicate electrochemical configuration                       |
|R9        |Photocatalysis and Photoelectrocatalysis experiments must indicate the light source used, including type of light, lamp, wavelength, irradiance and power|
|R10       |TiO2 catalysts must indicate crystal structure                                                                               |
|R11       |Medium reaction must be either gas or liquid                                                                                 |
|R12       |When reaction medium is liquid, the pH value must be indicated                                                               |
|R13       |Operation modes must be either batch or continuous                                                                           |
|R14       |When the operation mode is continuous, the spacial speed value must be indicated                                             |
|R15       |Each experiment may have one or more outputs                                                                                 |
|R16       |When the light source is not solar or solar simulation, the wavelength must be indicated                                     |
|R17       |An experiment may provide conversion metrics, quantum efficiency metrics and electrochemical metrics                         |
|CQ1       |How many experiments are reported per year?                                                                                  |
|CQ2       |How many experiments are reported per country?                                                                               |
|CQ3       |How many experiments are there per type of catalyst?                                                                         |
|CQ4       |Which articles have been published in the ACS NANO Journal in 2018 in volume 229?                                            |
|CQ5       |Which experiments use TiO2 as catalyst, liquid medium an produce H2 as output?                                               |
|CQ6       |Which experiments use TiO2 as catalyst, liquid medium and produce the most H2 output in µmol/gh?                             |


In [2]:
g = rdflib.Graph()
g.parse("result/solarchem-kg.nt")
print(len(g))

1904090


## R1: Articles have title, year, journal, issue, abstract, volume and author
Pending refinement with SemOpenAlex

In [5]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a bibo:Article ;
            bibo:doi ?doi ;
            bibo:abstract ?abs ;
            bibo:volume ?volume ;
            dc:title ?title ;
            bibo:issue ?issue ;
            dc:date ?date ;
            bibo:pageStart ?page .
    }

""")

for row in q_res:
    print(f"Number of articles with all properties: {row['count'].value}")

Number of articles with all properties: 415


## R2: Each article may refer to one or more experiments

In [6]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a bibo:Article ;
            solar:hasExperimentExecution ?expexec .
    }

""")

for row in q_res:
    print(f"Number of articles with experiments associated: {row['count'].value}")

Number of articles with experiments associated: 1096


## R3: Each experiment must have a catalyst

In [30]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT ?expexec ?catalyst
    WHERE {
        ?expexec prov:used ?input .
        ?input a solar:Input ;
            rdfs:label ?catalyst ;
            solar:hasRole solar:Catalyst .
    } LIMIT 10

""")

for row in q_res:
    print(f"{row.expexec} {row.catalyst}")

http://w3id.org/solar/i/ExpExec/6028 WO3
http://w3id.org/solar/i/ExpExec/4325 Ti-KIT-6
http://w3id.org/solar/i/ExpExec/515 TiO2
http://w3id.org/solar/i/ExpExec/4316 Ti-KIT-6
http://w3id.org/solar/i/ExpExec/2911 TiO2
http://w3id.org/solar/i/ExpExec/5587 g-C3N4
http://w3id.org/solar/i/ExpExec/3211 g-C3N4
http://w3id.org/solar/i/ExpExec/3848 BiOCl
http://w3id.org/solar/i/ExpExec/5895 SiC
http://w3id.org/solar/i/ExpExec/5488 TiO2


## R4: Each experiment may have additional inputs apart from the catalyst

In [31]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT DISTINCT  ?expexec ?inputlabel ?role
    WHERE {
        ?expexec prov:used ?input .
        ?input a solar:Input ;
            rdfs:label ?inputlabel ;
            solar:hasRole ?role .
        FILTER(?role != solar:Catalyst)
    } LIMIT 10

""")

for row in q_res:
    print(f"{row.expexec} {row.inputlabel} {row.role}")

http://w3id.org/solar/i/ExpExec/5126 [Ru(bpy)3]Cl2·6H2O http://w3id.org/solar/o/pc#Dye
http://w3id.org/solar/i/ExpExec/2196 Ru(bpy) http://w3id.org/solar/o/pc#Dye
http://w3id.org/solar/i/ExpExec/5991 [Ru(bpy)3]Cl2·6H2O http://w3id.org/solar/o/pc#Dye
http://w3id.org/solar/i/ExpExec/2566 Ru-Ru(bypy) http://w3id.org/solar/o/pc#Dye
http://w3id.org/solar/i/ExpExec/5476 [Ru(bpy)3]Cl2 http://w3id.org/solar/o/pc#Dye
http://w3id.org/solar/i/ExpExec/5137 [Ru(bpy)3]Cl2·6H2O http://w3id.org/solar/o/pc#Dye
http://w3id.org/solar/i/ExpExec/5858 [Ru(bpy)3]Cl2·6H2O http://w3id.org/solar/o/pc#Dye
http://w3id.org/solar/i/ExpExec/6124 2,2-bipyridine http://w3id.org/solar/o/pc#Dye
http://w3id.org/solar/i/ExpExec/1789 Ru(bpy) http://w3id.org/solar/o/pc#Dye
http://w3id.org/solar/i/ExpExec/6286 RhB http://w3id.org/solar/o/pc#Dye


## R5: Each experiment must specify surface area and band gap

In [32]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT DISTINCT ?expexec ?bgap ?bgapunit ?surfacearea ?saunit
    WHERE {
        ?expexec prov:used ?input .
        ?input a solar:Input ;
            solar:hasBandGap [ qudt:numericValue ?bgap; qudt:unit ?bgapunit ] ;
            solar:hasSurfaceArea [ qudt:numericValue ?surfacearea; qudt:unit ?saunit ] .
    } LIMIT 10

""")

for row in q_res:
    print(f"{row.expexec} Bandgap: {row.bgap} {row.bgapunit} | Surface area: {row.surfacearea} {row.saunit}")

http://w3id.org/solar/i/ExpExec/2939 Bandgap: 3.13 http://qudt.org/vocab/unit/EV | Surface area: 53.0 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/5568 Bandgap: 2.7 http://qudt.org/vocab/unit/EV | Surface area: 82.8 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/7715 Bandgap: 3.2 http://qudt.org/vocab/unit/EV | Surface area: 45.2 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/2413 Bandgap: 2.7 http://qudt.org/vocab/unit/EV | Surface area: 1149.4 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/2222 Bandgap: 3.4 http://qudt.org/vocab/unit/EV | Surface area: 240.0 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/2158 Bandgap: 3.2 http://qudt.org/vocab/unit/EV | Surface area: 46.9 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpExec/2588 Bandgap: 1.5 http://qudt.org/vocab/unit/EV | Surface area: 92.71 http://qudt.org/vocab/unit/M2-PER-GM
http://w3id.org/solar/i/ExpEx

## R6: An experiment must indicate operation conditions, including temperature, pressure, reaction time, medium and operation mode
Note: Medium and operation mode classes in onto, not in this KG's version

In [35]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition ?condition .
        ?condition a ?condition_type .
        VALUES ?condition_type {solar:TimeCondition solar:PressureCondition solar:TemperatureCondition}
    }
""")

for row in q_res:
    print(f"Number of articles with conditions: {row['count'].value}")

Number of articles with experiments associated: 6664


## R7: Photocatalysis experiments may also indicate type of reactor

In [37]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition ?condition .
        ?condition a solarpc:ReactorCondition .
    }
""")

for row in q_res:
    print(f"Number of articles with reactor conditions: {row['count'].value}")

Number of articles with reactor conditions: 3835


## R8: Electrocatalysis and Photoelectrocatalysis experiments may also indicate electrochemical configuration
No experiments so far with this condition, only from photocatalysis

In [38]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec solar:hasCondition ?condition .
        ?condition a solar:ElectrochemicalConfiguration .
    }
""")

for row in q_res:
    print(f"Number of articles with electrochemical configuration: {row['count'].value}")

Number of articles with electrochemical configuration: 0


## R9: Photocatalysis and Photoelectrocatalysis experiments must indicate the light source used, including type of light, lamp, wavelength, irradiance, illuminated area and power

In [33]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>
    PREFIX obi: <http://purl.obolibrary.org/obo/OBI_>

    SELECT DISTINCT ?expexec ?lamp ?wl ?irradiance ?power ?illum_area
    WHERE {
        ?expexec solar:hasLightSource ?light_source.
        ?light_source a obi:0400065 ;
            solar:hasLamp ?lamp ;
            solar:hasWavelength ?wl ;
            solar:hasIrradiance [qudt:numericValue ?irradiance] ;
            solar:hasPower [qudt:numericValue ?power] ;
            solar:hasIlluminatedArea [qudt:numericValue ?illum_area] .
            
    } LIMIT 5
""")

for row in q_res:
    print(f"{row.expexec}\n\tLamp: {row.lamp}\n\tWavelength: {row.wl}\n\tIrradiance: {row.irradiance}\n\tPower: {row.power}\n\tIlluminated Area: {row.illum_area}")

http://w3id.org/solar/i/ExpExec/3027
	Lamp: http://w3id.org/solar/o/core#XenonLamp
	Wavelength: http://w3id.org/solar/o/core#400-780WL
	Irradiance: 350.0
	Power: 500.0
	IlluminatedArea: 238.0
http://w3id.org/solar/i/ExpExec/3023
	Lamp: http://w3id.org/solar/o/core#XenonLamp
	Wavelength: http://w3id.org/solar/o/core#400-780WL
	Irradiance: 350.0
	Power: 500.0
	IlluminatedArea: 238.0
http://w3id.org/solar/i/ExpExec/3051
	Lamp: http://w3id.org/solar/o/core#XenonLamp
	Wavelength: http://w3id.org/solar/o/core#400-780WL
	Irradiance: 350.0
	Power: 500.0
	IlluminatedArea: 238.0
http://w3id.org/solar/i/ExpExec/3741
	Lamp: http://w3id.org/solar/o/core#XenonLamp
	Wavelength: http://w3id.org/solar/o/core#192-280WL
	Irradiance: 2700.0
	Power: 300.0
	IlluminatedArea: 368.0
http://w3id.org/solar/i/ExpExec/3041
	Lamp: http://w3id.org/solar/o/core#XenonLamp
	Wavelength: http://w3id.org/solar/o/core#400-780WL
	Irradiance: 350.0
	Power: 500.0
	IlluminatedArea: 238.0


## R10: TiO2 catalysts must indicate crystal structure

In [47]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT ?input ?crystal_str
    WHERE {
        ?input a solar:Input ;
            rdfs:label "TiO2" ;
            solar:hasRole solar:Catalyst ;
            solar:crystalStructure ?crystal_str .
    } LIMIT 10

""")

for row in q_res:
    print(f"{row.input}  {row.crystal_str}")

http://w3id.org/solar/i/Input/Catalyst/2828-TiO2  Anatase
http://w3id.org/solar/i/Input/Catalyst/4612-TiO2  Anatase
http://w3id.org/solar/i/Input/Catalyst/2063-TiO2  Anatase
http://w3id.org/solar/i/Input/Catalyst/201-TiO2  P25
http://w3id.org/solar/i/Input/Catalyst/5111-TiO2  P25
http://w3id.org/solar/i/Input/Catalyst/315-TiO2  P25
http://w3id.org/solar/i/Input/Catalyst/319-TiO2  Anatase
http://w3id.org/solar/i/Input/Catalyst/6173-TiO2  Anatase
http://w3id.org/solar/i/Input/Catalyst/446-TiO2  Mix
http://w3id.org/solar/i/Input/Catalyst/5022-TiO2  Anatase


## R11: Medium reaction must be either gas or liquid
When merged with the ontology

## R12: When reaction medium is liquid, the pH value must be indicated

In [52]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT ?expexec ?ph
    WHERE {
        ?expexec solar:hasCondition solar:LiquidMedium ;
            solar:hasCondition ?ph_cond .
        ?ph_cond a solar:pHCondition ;
            qudt:numericValue ?ph .
    } LIMIT 10
""")

for row in q_res:
    print(f"{row.expexec} pH: {row.ph}")

http://w3id.org/solar/i/ExpExec/3277 pH: 2.0
http://w3id.org/solar/i/ExpExec/5059 pH: 7.0
http://w3id.org/solar/i/ExpExec/6435 pH: 7.0
http://w3id.org/solar/i/ExpExec/5137 pH: 7.0
http://w3id.org/solar/i/ExpExec/3282 pH: 10.0
http://w3id.org/solar/i/ExpExec/4893 pH: 7.0
http://w3id.org/solar/i/ExpExec/6377 pH: 7.0
http://w3id.org/solar/i/ExpExec/6296 pH: 1.0
http://w3id.org/solar/i/ExpExec/5188 pH: 7.0
http://w3id.org/solar/i/ExpExec/5057 pH: 7.0


## R13: Operation modes must be either batch or continuous
When merged with the ontology

## R14: When the operation mode is continuous, the spacial speed value must be indicated


In [4]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT ?expexec ?restime ?unit
    WHERE {
        ?expexec solar:hasCondition solar:ContinuousMode ;
            solar:hasCondition ?restime_cond .
        ?restime_cond a solar:ResidenceTimeCondition ;
            qudt:numericValue ?restime;
            qudt:unit ?unit.
    } LIMIT 5
""")

for row in q_res:
    print(f"{row.expexec} Residence time: {row.restime} {row.unit}")

http://w3id.org/solar/i/ExpExec/31 Residence time: 83.3 http://qudt.org/vocab/unit/PER-MIN
http://w3id.org/solar/i/ExpExec/4917 Residence time: 0.24 http://qudt.org/vocab/unit/PER-MIN
http://w3id.org/solar/i/ExpExec/146 Residence time: 83.3 http://qudt.org/vocab/unit/PER-MIN
http://w3id.org/solar/i/ExpExec/529 Residence time: 30.0 http://qudt.org/vocab/unit/PER-MIN
http://w3id.org/solar/i/ExpExec/2866 Residence time: 0.28 http://qudt.org/vocab/unit/PER-MIN


## R15: Each experiment may have one or more outputs

In [6]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>
    PREFIX prov: <http://www.w3.org/ns/prov#>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?output a solar:Output ;
            prov:wasGeneratedBy ?expexec .
    }

""")

for row in q_res:
    print(f"Number of experiments with outputs: {row['count'].value}")

Number of experiments with outputs: 6664


## R16: When the light source is not solar or solar simulation, the wavelength must be indicated

In [ ]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>
    PREFIX obi: <http://purl.obolibrary.org/obo/OBI_>

    SELECT (COUNT (DISTINCT ?light_source) AS ?count)
    WHERE {
        ?light_source a ?type ;
            solar:hasWavelength ?wl .
        FILTER (?type != solar:SolarLightSource || ?type != solar:SolarSimulatorLightSource )
    } #LIMIT 10
""")

for row in q_res:
    print(f"Number of lightsources with all properties: {row['count'].value}")

## R17: An experiment may provide conversion metrics, quantum efficiency metrics and electrochemical metrics
Not yet in the KG

## CQ1: How many experiments are reported per year?

In [20]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT ?year (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?article a bibo:Article ;
            dc:date ?year ;
            solar:hasExperimentExecution ?expexec
    } GROUP BY ?year
    ORDER BY DESC (?year)

""")

for row in q_res:
    print(f"Experiments in {row.year}: {row['count'].value}")

Experiments in 2022: 284
Experiments in 2021: 1012
Experiments in 2020: 1303
Experiments in 2019: 334
Experiments in 2018: 500
Experiments in 2017: 692
Experiments in 2016: 368
Experiments in 2015: 543
Experiments in 2014: 432
Experiments in 2013: 349
Experiments in 2012: 209
Experiments in 2011: 203
Experiments in 2010: 43
Experiments in 2009: 38
Experiments in 2008: 46
Experiments in 2007: 53
Experiments in 2006: 15
Experiments in 2005: 20
Experiments in 2004: 19
Experiments in 2003: 22
Experiments in 2002: 18
Experiments in 2001: 12
Experiments in 2000: 11
Experiments in 1999: 8
Experiments in 1998: 21
Experiments in 1997: 30
Experiments in 1996: 2
Experiments in 1995: 17
Experiments in 1994: 20
Experiments in 1993: 7
Experiments in 1992: 16
Experiments in 1991: 1
Experiments in 1990: 2
Experiments in 1987: 1
Experiments in 1984: 6
Experiments in 1979: 7


## CQ2: How many experiments are reported per country?
Not yet in the KG

## CQ3: How many experiments are there per type of catalyst?

In [27]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT ?catalyst (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec prov:used ?input .
        ?input solar:hasRole solar:Catalyst ;
            rdfs:label ?catalyst .
    } GROUP BY ?catalyst
    ORDER BY DESC (?count)

""")

for row in q_res:
    print(f"Experiments with {row.catalyst}: {row['count'].value}")

Experiments with TiO2: 2131
Experiments with g-C3N4: 745
Experiments with Ti-KIT-6: 201
Experiments with ZnO: 182
Experiments with CeO2: 160
Experiments with ZnS: 138
Experiments with CdS: 112
Experiments with BiOCl: 85
Experiments with Cu2O: 83
Experiments with Bi2WO6: 82
Experiments with CsPbBr3: 66
Experiments with WO3: 65
Experiments with BiVO4: 64
Experiments with LDH: 55
Experiments with MgIn2S4: 55
Experiments with Ga2O3: 54
Experiments with In2O3: 41
Experiments with CuAlGaO4: 40
Experiments with AgBr: 40
Experiments with ZrO2: 38
Experiments with K2Ti6O13: 38
Experiments with ZIF-9: 38
Experiments with Pd(bpy)-F23: 34
Experiments with Zn2GeO4: 34
Experiments with InTaO4: 31
Experiments with UIO-66-NH2(Zr): 31
Experiments with BiOBr: 30
Experiments with Bi3O4Cl: 30
Experiments with SrTiO3: 29
Experiments with MoS2: 29
Experiments with MoO3-x: 28
Experiments with Carbon Nitride: 26
Experiments with C3N4: 26
Experiments with ZnIn2S4: 25
Experiments with [Zn1.5Cu1.5Ga(OH)8]2(CO3)2

## CQ4: Which articles have been published in the ACS NANO Journal in 2018 in volume 229?
Not in current KG

## CQ5: Which experiments use TiO2 as catalyst, liquid medium an produce H2 as output?
Muy lenta :( y a la de abajo le va a pasar lo mismo

In [31]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec prov:used ?input ;
            solar:hasCondition solar:LiquidMedium .
            
        ?input solar:hasRole solar:Catalyst ;
            rdfs:label "TiO2" .
    
        ?output rdfs:label "H2" ;
            prov:wasGeneratedBy ?expexec .
    }
""")

for row in q_res:
    print(f"Experiments: {row['count'].value}")

KeyboardInterrupt: 

## CQ6: Which experiments use TiO2 as catalyst, liquid medium and produce the most H2 output in µmol/gh?

In [ ]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT (COUNT (DISTINCT ?expexec) AS ?count)
    WHERE {
        ?expexec prov:used ?input ;
            solar:hasCondition solar:LiquidMedium .
            
        ?input solar:hasRole solar:Catalyst ;
            rdfs:label "TiO2" .
    
        ?output rdfs:label "H2" ;
            qudt:unit unit:MicroMOL-PER-GM-HR ;
            prov:wasGeneratedBy ?expexec .
    }
""")

for row in q_res:
    print(f"Experiments: {row['count'].value}")